# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        img -= np.mean(img)
        img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return images
    #return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [7]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print (Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.4)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [8]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 10.9458 Disc Loss: 1.32811 Q Losses: [0.21357179, 2.31915]
epoch:0 batch_done:2 Gen Loss: 3.15101 Disc Loss: 0.849551 Q Losses: [0.16497548, 2.2869556]
epoch:0 batch_done:3 Gen Loss: 8.27954 Disc Loss: 3.4309 Q Losses: [0.1441353, 2.2921443]
epoch:0 batch_done:4 Gen Loss: 8.21519 Disc Loss: 0.43 Q Losses: [0.13175231, 2.2809346]
epoch:0 batch_done:5 Gen Loss: 4.128 Disc Loss: 0.610781 Q Losses: [0.12942418, 2.2464159]
epoch:0 batch_done:6 Gen Loss: 12.6174 Disc Loss: 1.59001 Q Losses: [0.1272292, 2.2679677]
epoch:0 batch_done:7 Gen Loss: 9.83093 Disc Loss: 0.618597 Q Losses: [0.11024691, 2.2857561]
epoch:0 batch_done:8 Gen Loss: 5.677 Disc Loss: 0.439469 Q Losses: [0.098542802, 2.2347558]
epoch:0 batch_done:9 Gen Loss: 17.6245 Disc Loss: 1.24073 Q Losses: [0.13354966, 2.2248385]
epoch:0 batch_done:10 Gen Loss: 11.9186 Disc Loss: 1.31279 Q Losses: [0.12866673, 2.2379053]
epoch:0 batch_done:11 Gen Loss: 12.5128 Disc Loss: 1.22794 Q Losses: [0.14398798, 2.25

epoch:0 batch_done:89 Gen Loss: 15.6827 Disc Loss: 0.498527 Q Losses: [0.049135156, 1.0110338]
epoch:0 batch_done:90 Gen Loss: 15.6471 Disc Loss: 0.263009 Q Losses: [0.04556901, 0.99888504]
epoch:0 batch_done:91 Gen Loss: 10.7038 Disc Loss: 0.337243 Q Losses: [0.060276411, 0.93649578]
epoch:0 batch_done:92 Gen Loss: 6.03509 Disc Loss: 0.0187467 Q Losses: [0.060645178, 0.93784273]
epoch:0 batch_done:93 Gen Loss: 4.53667 Disc Loss: 0.0577606 Q Losses: [0.052497506, 0.94354129]
epoch:0 batch_done:94 Gen Loss: 35.8428 Disc Loss: 3.87366 Q Losses: [0.057408828, 0.91440213]
epoch:0 batch_done:95 Gen Loss: 32.6584 Disc Loss: 5.11046 Q Losses: [0.042902149, 0.97585893]
epoch:0 batch_done:96 Gen Loss: 15.9415 Disc Loss: 2.25648 Q Losses: [0.062953189, 0.91213387]
epoch:0 batch_done:97 Gen Loss: 4.705 Disc Loss: 0.00280548 Q Losses: [0.051126387, 0.92785966]
epoch:0 batch_done:98 Gen Loss: 26.379 Disc Loss: 2.02586 Q Losses: [0.051883452, 0.93642032]
epoch:0 batch_done:99 Gen Loss: 29.0913 Disc 

epoch:0 batch_done:174 Gen Loss: 2.74783 Disc Loss: 0.310143 Q Losses: [0.044418976, 0.44240654]
epoch:0 batch_done:175 Gen Loss: 38.1285 Disc Loss: 1.48185 Q Losses: [0.03633707, 0.41740835]
epoch:0 batch_done:176 Gen Loss: 35.6721 Disc Loss: 5.218 Q Losses: [0.05071082, 0.4088586]
epoch:0 batch_done:177 Gen Loss: 24.0643 Disc Loss: 1.56083 Q Losses: [0.027534572, 0.3891032]
epoch:0 batch_done:178 Gen Loss: 13.3361 Disc Loss: 0.0578509 Q Losses: [0.036547113, 0.42690682]
epoch:0 batch_done:179 Gen Loss: 4.95272 Disc Loss: 0.00168932 Q Losses: [0.040757176, 0.37108791]
epoch:0 batch_done:180 Gen Loss: 24.518 Disc Loss: 0.751862 Q Losses: [0.042871885, 0.39081991]
epoch:0 batch_done:181 Gen Loss: 27.7075 Disc Loss: 0.627005 Q Losses: [0.035064097, 0.37766346]
epoch:0 batch_done:182 Gen Loss: 24.586 Disc Loss: 0.19087 Q Losses: [0.030601291, 0.38440567]
epoch:0 batch_done:183 Gen Loss: 15.0721 Disc Loss: 0.666043 Q Losses: [0.024242125, 0.34801346]
epoch:0 batch_done:184 Gen Loss: 7.0631

epoch:1 batch_done:52 Gen Loss: 19.2331 Disc Loss: 0.0937015 Q Losses: [0.035721712, 0.23643807]
epoch:1 batch_done:53 Gen Loss: 15.2304 Disc Loss: 0.0622543 Q Losses: [0.022539319, 0.22322707]
epoch:1 batch_done:54 Gen Loss: 12.1318 Disc Loss: 0.0187297 Q Losses: [0.020993453, 0.22452329]
epoch:1 batch_done:55 Gen Loss: 9.1564 Disc Loss: 0.0052344 Q Losses: [0.022505116, 0.21044081]
epoch:1 batch_done:56 Gen Loss: 5.37625 Disc Loss: 0.0155893 Q Losses: [0.020974025, 0.20768592]
epoch:1 batch_done:57 Gen Loss: 6.14202 Disc Loss: 0.0450611 Q Losses: [0.018717833, 0.21923485]
epoch:1 batch_done:58 Gen Loss: 10.3893 Disc Loss: 0.0923494 Q Losses: [0.025330933, 0.21828142]
epoch:1 batch_done:59 Gen Loss: 10.1261 Disc Loss: 0.0206376 Q Losses: [0.02169079, 0.24335001]
epoch:1 batch_done:60 Gen Loss: 8.21619 Disc Loss: 0.17165 Q Losses: [0.01834058, 0.21824226]
epoch:1 batch_done:61 Gen Loss: 5.74229 Disc Loss: 0.0720023 Q Losses: [0.021183487, 0.20109616]
epoch:1 batch_done:62 Gen Loss: 43.

epoch:1 batch_done:137 Gen Loss: 5.74755 Disc Loss: 0.110352 Q Losses: [0.014618982, 0.12081273]
epoch:1 batch_done:138 Gen Loss: 6.00592 Disc Loss: 0.0502774 Q Losses: [0.013988306, 0.11840292]
epoch:1 batch_done:139 Gen Loss: 6.00685 Disc Loss: 0.0445325 Q Losses: [0.010495804, 0.13367563]
epoch:1 batch_done:140 Gen Loss: 5.81746 Disc Loss: 0.0540749 Q Losses: [0.019133648, 0.13226074]
epoch:1 batch_done:141 Gen Loss: 44.373 Disc Loss: 1.79584 Q Losses: [0.014456763, 0.14533398]
epoch:1 batch_done:142 Gen Loss: 41.851 Disc Loss: 6.69951 Q Losses: [0.015682438, 0.12732081]
epoch:1 batch_done:143 Gen Loss: 30.9952 Disc Loss: 1.6108 Q Losses: [0.014905905, 0.12762889]
epoch:1 batch_done:144 Gen Loss: 19.6258 Disc Loss: 0.0302921 Q Losses: [0.01333693, 0.14925128]
epoch:1 batch_done:145 Gen Loss: 7.7577 Disc Loss: 0.00119778 Q Losses: [0.024790194, 0.14021328]
epoch:1 batch_done:146 Gen Loss: 38.334 Disc Loss: 1.12439 Q Losses: [0.016198499, 0.13767809]
epoch:1 batch_done:147 Gen Loss: 4

...

epoch:53 batch_done:110 Gen Loss: 10.6523 Disc Loss: 0.00137949 Q Losses: [0.050953105, 0.054620817]
epoch:53 batch_done:111 Gen Loss: 10.2425 Disc Loss: 0.0169096 Q Losses: [0.040582709, 0.043327864]
epoch:53 batch_done:112 Gen Loss: 8.61347 Disc Loss: 0.0305762 Q Losses: [0.071015492, 0.040432762]
epoch:53 batch_done:113 Gen Loss: 7.03137 Disc Loss: 0.0067656 Q Losses: [0.056881644, 0.13883869]
epoch:53 batch_done:114 Gen Loss: 9.30694 Disc Loss: 0.00566731 Q Losses: [0.02564279, 0.29287386]
epoch:53 batch_done:115 Gen Loss: 12.9177 Disc Loss: 0.0366785 Q Losses: [0.044041403, 0.44404876]
epoch:53 batch_done:116 Gen Loss: 17.8216 Disc Loss: 0.00166448 Q Losses: [0.055630066, 1.2745867]
epoch:53 batch_done:117 Gen Loss: 22.5769 Disc Loss: 1.02728 Q Losses: [0.026992224, 0.71011859]
epoch:53 batch_done:118 Gen Loss: 25.8887 Disc Loss: 0.0551912 Q Losses: [0.074110724, 0.42314431]
epoch:53 batch_done:119 Gen Loss: 28.081 Disc Loss: 1.29633 Q Losses: [0.044753663, 0.6209774]
epoch:53 bat

epoch:53 batch_done:202 Gen Loss: nan Disc Loss: nan Q Losses: [0.16421513, 2.3033137]
epoch:53 batch_done:203 Gen Loss: nan Disc Loss: nan Q Losses: [0.16355491, 2.3023229]
epoch:53 batch_done:204 Gen Loss: nan Disc Loss: nan Q Losses: [0.14653881, 2.3022079]
epoch:53 batch_done:205 Gen Loss: nan Disc Loss: nan Q Losses: [0.16370302, 2.3043251]
epoch:53 batch_done:206 Gen Loss: nan Disc Loss: nan Q Losses: [0.15306708, 2.3013721]
epoch:53 batch_done:207 Gen Loss: nan Disc Loss: nan Q Losses: [0.16210362, 2.3014913]
epoch:54 batch_done:1 Gen Loss: nan Disc Loss: nan Q Losses: [0.14006335, 2.3023086]
epoch:54 batch_done:2 Gen Loss: nan Disc Loss: nan Q Losses: [0.1915018, 2.3023834]
epoch:54 batch_done:3 Gen Loss: nan Disc Loss: nan Q Losses: [0.16490096, 2.3033545]
epoch:54 batch_done:4 Gen Loss: nan Disc Loss: nan Q Losses: [0.20159678, 2.3029675]
epoch:54 batch_done:5 Gen Loss: nan Disc Loss: nan Q Losses: [0.14330846, 2.3014235]
epoch:54 batch_done:6 Gen Loss: nan Disc Loss: nan Q L

epoch:54 batch_done:92 Gen Loss: nan Disc Loss: nan Q Losses: [0.14118747, 2.3022339]
epoch:54 batch_done:93 Gen Loss: nan Disc Loss: nan Q Losses: [0.17215349, 2.2998185]
epoch:54 batch_done:94 Gen Loss: nan Disc Loss: nan Q Losses: [0.16564958, 2.3024168]
epoch:54 batch_done:95 Gen Loss: nan Disc Loss: nan Q Losses: [0.20762619, 2.3017297]
epoch:54 batch_done:96 Gen Loss: nan Disc Loss: nan Q Losses: [0.15842097, 2.3026004]
epoch:54 batch_done:97 Gen Loss: nan Disc Loss: nan Q Losses: [0.15721267, 2.3025224]
epoch:54 batch_done:98 Gen Loss: nan Disc Loss: nan Q Losses: [0.16903631, 2.3015409]
epoch:54 batch_done:99 Gen Loss: nan Disc Loss: nan Q Losses: [0.16110626, 2.3034065]
epoch:54 batch_done:100 Gen Loss: nan Disc Loss: nan Q Losses: [0.16477408, 2.3053672]
epoch:54 batch_done:101 Gen Loss: nan Disc Loss: nan Q Losses: [0.14904168, 2.3012052]
epoch:54 batch_done:102 Gen Loss: nan Disc Loss: nan Q Losses: [0.18884476, 2.3034091]
epoch:54 batch_done:103 Gen Loss: nan Disc Loss: na

epoch:54 batch_done:188 Gen Loss: nan Disc Loss: nan Q Losses: [0.20218526, 2.3048277]
epoch:54 batch_done:189 Gen Loss: nan Disc Loss: nan Q Losses: [0.19393741, 2.3038428]
epoch:54 batch_done:190 Gen Loss: nan Disc Loss: nan Q Losses: [0.1667127, 2.2997355]
epoch:54 batch_done:191 Gen Loss: nan Disc Loss: nan Q Losses: [0.16181073, 2.3024971]
epoch:54 batch_done:192 Gen Loss: nan Disc Loss: nan Q Losses: [0.16142169, 2.3013883]
epoch:54 batch_done:193 Gen Loss: nan Disc Loss: nan Q Losses: [0.17615563, 2.3038697]
epoch:54 batch_done:194 Gen Loss: nan Disc Loss: nan Q Losses: [0.15393043, 2.3038554]
epoch:54 batch_done:195 Gen Loss: nan Disc Loss: nan Q Losses: [0.14244261, 2.3015506]
epoch:54 batch_done:196 Gen Loss: nan Disc Loss: nan Q Losses: [0.15890792, 2.3021712]
epoch:54 batch_done:197 Gen Loss: nan Disc Loss: nan Q Losses: [0.16138652, 2.3021824]
epoch:54 batch_done:198 Gen Loss: nan Disc Loss: nan Q Losses: [0.15038106, 2.3019826]
epoch:54 batch_done:199 Gen Loss: nan Disc L

KeyboardInterrupt: 

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 